In [ ]:
from nsepython import *   
import datetime

symbol = "NIFTY 50"
days = 20
end_date = datetime.datetime.now().strftime("%d-%b-%Y")
end_date = str(end_date)

start_date = (datetime.datetime.now()- datetime.timedelta(days=days)).strftime("%d-%b-%Y")
start_date = str(start_date)

df2=index_history("NIFTY 50",start_date,end_date)
df2["daily_change"]=df2["CLOSE"].astype(float) / df2["OPEN"].astype(float) #.pct_change()
df2=df2[['Index Name','HistoricalDate','daily_change', 'OPEN','HIGH','LOW', 'CLOSE']]
df2 = df2.iloc[1: , :]
print(df2)

In [ ]:
nse_events()

In [ ]:
print(nse_fiidii())

In [ ]:
print(nse_get_fno_lot_sizes()) # mode='pandas'

In [ ]:
# key = "NIFTY"
# key = "BANKNIFTY"
# key = "SME"
# key = "FO"
# key = "OTHERS"
# key = "ALL"
import pandas as pd

def nse_preopen(key="NIFTY"):
    returnTo = []
    payload = nsefetch("https://www.nseindia.com/api/market-data-pre-open?key="+key+"")   
    dataList = payload['data']
    for d in dataList:
      
        obj = {}
        obj['lastUpdateTime'] = d['detail']['preOpenMarket']['lastUpdateTime']
        obj['symbol'] = d['metadata']['symbol']
        obj['lastPrice'] = d['metadata']['lastPrice']
        obj['change'] = d['metadata']['change']
        obj['pChange'] = d['metadata']['pChange']
        obj['previousClose'] = d['metadata']['previousClose']
        obj['yearHigh'] = d['metadata']['yearHigh']
        obj['yearLow'] = d['metadata']['yearLow']
        returnTo.append(obj)
    return returnTo

nse_preopen("NIFTY")

In [ ]:

pre_open_df = pd.DataFrame(nse_preopen("NIFTY"))
pre_open_df

In [ ]:
positions = nsefetch('https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O')
# print(positions)
postions_df = pd.DataFrame(positions['data'])
postions_df.columns.values

postions_df[['symbol', 'identifier', 'series', 'open', 'dayHigh', 'dayLow',
       'lastPrice', 'previousClose', 'change', 'pChange', 'yearHigh', 'yearLow', 'nearWKH', 'nearWKL', 'perChange365d',
       'date365dAgo', 'chart365dPath', 'date30dAgo', 'perChange30d']]

In [ ]:
def nse_optionchain_ltp(payload,strikePrice,optionType,inp=0,intent=""):
    expiryDate=payload['records']['expiryDates'][inp]
    for x in range(len(payload['records']['data'])):
      if((payload['records']['data'][x]['strikePrice']==strikePrice) & (payload['records']['data'][x]['expiryDate']==expiryDate)):
          if(intent==""): 
              return payload['records']['data'][x][optionType]['lastPrice']
          if(intent=="sell"): 
              return payload['records']['data'][x][optionType]['bidprice']
          if(intent=="buy"): 
              return payload['records']['data'][x][optionType]['askPrice']
          
payload=nse_optionchain_scrapper('BANKNIFTY')
print(nse_optionchain_ltp(payload,41000,"CE",0,"sell"))

In [ ]:
print(nse_marketStatus())

In [ ]:
print(nse_fiidii())


In [ ]:
index_df = nse_index()
index_df = index_df[['timeVal', 'indexName', 'percChange',  'last', 'previousClose', 'high' , 'low', 'open', 'yearHigh', 'yearLow']]
index_df = index_df[((index_df['indexName'] == 'NIFTY 50') |
                      (index_df['indexName'] == 'NIFTY BANK') |
                     (index_df['indexName'] == 'NIFTY NEXT 50') | 
                    
                     (index_df['indexName'] == 'INDIA VIX')
                     )]
index_df

In [ ]:
nse_index()

In [ ]:
print(nse_get_advances_declines())
print(nse_get_advances_declines().columns.values)


In [ ]:
indiavix()
print(index_info("INDIA VIX"))


In [ ]:
S0 = 34950.60
X = 35000.00
σ = 14.72
t = 3
call_theta,put_theta,call_premium,put_premium,call_delta,put_delta,gamma,vega,call_rho,put_rho=black_scholes_dexter(S0,X,t,σ="",r=10,q=0.0,td=365)
print(call_theta)
print(put_theta)
print(call_premium)
print(put_premium)
print(call_delta)
print(put_delta)
print(gamma)
print(vega)
print(call_rho)
print(put_rho)

In [ ]:
print(get_bhavcopy("04-06-2021"))


In [ ]:
nse_preopen_movers(key="FO")


In [ ]:
movers=nse_preopen_movers("NIFTY")
# print(gainers)
print(movers)

In [ ]:
nse_most_active(type="securities",sort="volume")
# type = "securities" (default)
# type = "etf"
# type = "sme"

# sort = "volume"
# sort = "value" (default)


In [ ]:
from nsepython import *  
from datetime import datetime
import pandas as pd

underlying_option_symbol = "BANKNIFTY" 
script_master = pd.read_csv('./scrip_master.csv')

# BANKNIFTY20OCT2232000PE
def getFNOToken(underlying_option_symbol, index, strike, option_type):
    underlying_option_expiry_list = expiry_list(underlying_option_symbol)
    date_object = datetime.strptime(underlying_option_expiry_list[index], '%d-%b-%Y')
    str_date = date_object.strftime('%d%b%Y').replace('20', '')
    str_date = str_date.upper()
    symbol_to = underlying_option_symbol+str_date+strike+option_type
    df = script_master[script_master['symbol'] == symbol_to]
    return df[['symbol', 'token', 'name', 'expiry', 'strike', 'lotsize']]
    
getFNOToken("BANKNIFTY", 0, "41500","CE")


In [5]:

from nsepython import *
def get_atm_strike(symbol):
    symbol = symbol.upper()
    option_chain_json = nse_optionchain_scrapper(symbol)
    data = option_chain_json['filtered']['data']
    ltp = data[0]['PE']['underlyingValue']
    strike_price_list = [x['strikePrice'] for x in data]
    atm_strike = sorted([[round(abs(ltp-i),2),i] for i in strike_price_list])[0][1]
    return atm_strike

def get_pe_ce_price(symbol,atm_strike):
    symbol = symbol.upper()
    option_chain = nse_optionchain_scrapper(symbol)
    for dictt in option_chain['filtered']['data']:
        if dictt['strikePrice'] ==atm_strike:
            pe_price = dictt['PE']['askPrice']
            ce_price = dictt['CE']['askPrice']
            return pe_price,ce_price
        
atm_strike = get_atm_strike("NIFTY")
get_pe_ce_price("NIFTY", atm_strike)

(65.95, 71)

In [8]:
oi_data, ltp, crontime = oi_chain_builder("NIFTY")
ltp

C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oi_data = oi_data.append(oi_row, ignore_index=True)
C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oi_data = oi_data.append(oi_row, ignore_index=True)
C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oi_data = oi_data.append(oi_row, ignore_index=True)
C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

17656.35